Стартовая настроечка не нужна, смотрим всё сразу

In [1]:
Persons = {
    "Артем": {
        "Док": "111-111-111 11",  # СНИЛС
        "Спец": {
            1: {"ОКСО": "09.03.01 (бюджет)", "Балл": 250, "Места": 101,
                "Ссылка": "https://priem.bmstu.ru/lists/upload/enrollees/first/moscow-1/09.03.01.pdf"},
            2: {"ОКСО": "09.03.03 (бюджет)", "Балл": 250, "Места": 84,
                "Ссылка": "https://priem.bmstu.ru/lists/upload/enrollees/first/moscow-1/09.03.03.pdf"},
            3: {"ОКСО": "10.05.01 (бюджет)", "Балл": 225, "Места": 7,
                "Ссылка": "https://priem.bmstu.ru/lists/upload/enrollees/first/moscow-1/10.05.01.pdf"},
            4: {"ОКСО": "09.03.01 (платка)", "Балл": 250, "Места": 184,
                "Ссылка": "https://priem.bmstu.ru/lists/upload/enrollees/first/moscow-0/09.03.01.pdf"},
            5: {"ОКСО": "09.03.03 (платка)", "Балл": 250, "Места": 85,
                "Ссылка": "https://priem.bmstu.ru/lists/upload/enrollees/first/moscow-0/09.03.03.pdf"},
            6: {"ОКСО": "10.05.01 (платка)", "Балл": 225, "Места": 39,
                "Ссылка": "https://priem.bmstu.ru/lists/upload/enrollees/first/moscow-0/10.05.01.pdf"}
        }
    },
    "Маша": {
        "Док": "222-222-222 22",  # СНИЛС
        "Спец": {
            1: {"ОКСО": "01.03.04 (бюджет)", "Балл": 233, "Места": 69,
                "Ссылка": "https://priem.bmstu.ru/lists/upload/enrollees/first/moscow-1/01.03.04.pdf"},
            2: {"ОКСО": "09.03.01 (бюджет)", "Балл": 233, "Места": 101,
                "Ссылка": "https://priem.bmstu.ru/lists/upload/enrollees/first/moscow-1/09.03.01.pdf"},
            3: {"ОКСО": "12.03.02 (бюджет)", "Балл": 233, "Места": 17,
                "Ссылка": "https://priem.bmstu.ru/lists/upload/enrollees/first/moscow-1/12.03.02.pdf"}
        }
    }
}
to_budget = {}
# Persons["Маша"]["Спец"]

In [2]:
import pandas as pd
import numpy as np
import re
from tabula import read_pdf # если ошибка, то pip install tabula-py

Парсим все данные по конкурсу на наши специальности

In [3]:
df_concurs = {}
for Person in Persons:
    for Spec in Persons[Person]["Спец"]:
        idx = Person+str(Spec)
#         print(idx)
        bmstu_path = Persons[Person]["Спец"][Spec]["Ссылка"]
        df = read_pdf(bmstu_path, pages='all')
        frames = []
        for i in range(len(df)):
            first_valid = 0
            df_tmp = df[i]
            try:
                scip_flag = (df[i].loc[0][0] == "No" or np.isnan(df[i].loc[0][0]))
            except Exception as e:
                scip_flag = True
        #     print("first_item=", df[i].loc[0][0], "len_df=",len(df[i]))
            if scip_flag:
                for j in range(len(df[i])):
                    try:
                        if df[i].loc[j][0].isnumeric():
                            if first_valid == 0:
                                first_valid = j
                    except Exception as e:
                       str1 = "we have found an exception"
        #     print("first_valid=",first_valid)
            df_tmp.columns = df[3].columns
            df_tmp = df_tmp[first_valid:]
            df_tmp = df_tmp.reset_index(drop=True)
            frames.append(df_tmp)
        df_concurs[idx] = pd.concat(frames, sort=False)
        df_concurs[idx].set_axis(['No', 'СНИЛС', 'Общ', 'Балл', 'Балл без ИД',
        df_concurs[idx].columns[5].replace("\r", "/"),
        df_concurs[idx].columns[6].replace("\r", "/"),
        df_concurs[idx].columns[7].replace("\r", "/"),
        'ИД', 'ПП', 'Ориг.', 'Согл.', 'Прогноз'], axis=1, inplace=True)
        df_concurs[idx] = df_concurs[idx].reset_index(drop=True)
# df_concurs["Маша3"]

In [4]:
df_concurs_head = {}
for Person in Persons:
    for Spec in Persons[Person]["Спец"]:
        idx = Person+str(Spec)
#         print(idx)
        num = df_concurs[idx][df_concurs[idx]["СНИЛС"] == Persons[Person]["Док"]].index.values[0] + 1
        print(
            Person + ", " + Persons[Person]["Спец"][Spec]["ОКСО"] + ": \033[1m\033[34m"
            + str(num)
            + "\033[0m из "
            + str(len(df_concurs[idx]))
            + " участвующих в этом конкурсе вообще"
        )
        df_concurs_head[idx] = df_concurs[idx][:num]
# df_concurs_head["Маша3"]

Артем, 09.03.01 (бюджет): 1525 из 2626 участвующих в этом конкурсе вообще
Артем, 09.03.03 (бюджет): 1259 из 2054 участвующих в этом конкурсе вообще
Артем, 10.05.01 (бюджет): 337 из 504 участвующих в этом конкурсе вообще
Артем, 09.03.01 (платка): 511 из 1384 участвующих в этом конкурсе вообще
Артем, 09.03.03 (платка): 386 из 1090 участвующих в этом конкурсе вообще
Артем, 10.05.01 (платка): 109 из 253 участвующих в этом конкурсе вообще
Маша, 01.03.04 (бюджет): 867 из 1018 участвующих в этом конкурсе вообще
Маша, 09.03.01 (бюджет): 1893 из 2626 участвующих в этом конкурсе вообще
Маша, 12.03.02 (бюджет): 251 из 351 участвующих в этом конкурсе вообще


Не по делу, а лишь только любопытства ради: сколько всего заплатили (вместе  теми, кто еще не принес Оригинал)

In [5]:
df_concurs_agree = {}
mark_pass = {}
for Person in ["Артем"]:
    for Spec in [4,5,6]:
        idx = Person+str(Spec)
        df_concurs_agree[idx] = df_concurs[idx][
#             (df_concurs[idx]["Ориг."] == "Да") & 
            (df_concurs[idx]["Согл."] == "Да")
        ]
        df_concurs_agree[idx] = df_concurs_agree[idx].reset_index(drop=True)
        place_pass_row = df_concurs_agree[idx].iloc[min(Persons[Person]["Спец"][Spec]["Места"], len(df_concurs_agree[idx]))-1]
        mark_pass[idx] = place_pass_row["Балл"]
        print(
            Persons[Person]["Спец"][Spec]["ОКСО"] + ": мест " + str(Persons[Person]["Спец"][Spec]["Места"]) + 
            ", согласий/оплат \033[1m\033[34m" + str(len(df_concurs_agree[idx])) + "\033[0m"
        )

09.03.01 (платка): мест 184, согласий/оплат 172
09.03.03 (платка): мест 85, согласий/оплат 83
10.05.01 (платка): мест 39, согласий/оплат 27


Выбираем тех, кто рассматривается к зачислению (только Оригинал + Согласие)

In [6]:
df_concurs_agree = {}
mark_pass = {}
for Person in Persons:
    for Spec in Persons[Person]["Спец"]:
        idx = Person+str(Spec)
#         print(idx)
        df_concurs_agree[idx] = df_concurs[idx][
            (df_concurs[idx]["Ориг."] == "Да")
            & (df_concurs[idx]["Согл."] == "Да")
        ]
        df_concurs_agree[idx] = df_concurs_agree[idx].reset_index(drop=True)
        place_pass_row = df_concurs_agree[idx].iloc[min(Persons[Person]["Спец"][Spec]["Места"], len(df_concurs_agree[idx]))-1]
        mark_pass[idx] = place_pass_row["Балл"]
# df_concurs_agree["Маша3"]

Дальше крутим-вертим просто любопытства ради, посмотреть у кого были какие запасные варианты, а кто сразу струсил и заплатил

In [ ]:
check = pd.merge(
    left=df_concurs_agree["Артем1"][["СНИЛС", "Балл", "Ориг.", "Согл.", "Прогноз"]], 
    right=df_concurs["Артем4"][["СНИЛС", "Ориг.", "Согл.", "Прогноз"]], 
    how="left", 
    on="СНИЛС", 
)
check = check.fillna("")
print("Бюджет -> Платка")
print("09.03.01,", "согласий", len(check))
check[101:126]

In [ ]:
check = pd.merge(
    left=df_concurs_agree["Артем4"][["СНИЛС", "Балл", "Ориг.", "Согл.", "Прогноз"]], 
    right=df_concurs["Артем1"][["СНИЛС", "Ориг.", "Согл.", "Прогноз"]], 
    how="left", 
    on="СНИЛС", 
)
check = check.fillna("")
to_budget["Артем4"] = check["СНИЛС"][check["Прогноз_y"] != ""]
# print(to_budget)
print("Платка -> Бюджет")
print("09.03.01", "согласий", len(check))
check.head(25)

In [ ]:
check = pd.merge(
    left=df_concurs_agree["Артем2"][["СНИЛС", "Балл", "Ориг.", "Согл.", "Прогноз"]], 
    right=df_concurs["Артем5"][["СНИЛС", "Ориг.", "Согл.", "Прогноз"]], 
    how="left", 
    on="СНИЛС", 
)
check = check.fillna("")
print("Бюджет -> Платка")
print("09.03.03,", "согласий", len(check))
check[84:94]

In [ ]:
check = pd.merge(
    left=df_concurs_agree["Артем5"][["СНИЛС", "Балл", "Ориг.", "Согл.", "Прогноз"]], 
    right=df_concurs["Артем2"][["СНИЛС", "Ориг.", "Согл.", "Прогноз"]], 
    how="left", 
    on="СНИЛС", 
)
check = check.fillna("")
to_budget["Артем5"] = check["СНИЛС"][check["Прогноз_y"] != ""]
# print(to_budget)
print("Платка -> Бюджет")
print("09.03.03,", "согласий", len(check))
check.head(10)

In [ ]:
check = pd.merge(
    left=df_concurs_agree["Артем3"][["СНИЛС", "Балл", "Ориг.", "Согл.", "Прогноз"]], 
    right=df_concurs["Артем6"][["СНИЛС", "Ориг.", "Согл.", "Прогноз"]], 
    how="left", 
    on="СНИЛС", 
)
check = check.fillna("")
print("Бюджет -> Платка")
print("10.05.01,", "согласий", len(check))
check[7:17]

In [ ]:
check = pd.merge(
    left=df_concurs_agree["Артем6"][["СНИЛС", "Балл", "Ориг.", "Согл.", "Прогноз"]], 
    right=df_concurs["Артем3"][["СНИЛС", "Ориг.", "Согл.", "Прогноз"]], 
    how="left", 
    on="СНИЛС", 
)
check = check.fillna("")
to_budget["Артем6"] = check["СНИЛС"][check["Прогноз_y"] != ""]
# print(to_budget)
print("Платка -> Бюджет")
print("10.05.01,", "согласий", len(check))
check.head(10)

In [ ]:
for Person in Persons:
    for Spec in Persons[Person]["Спец"]:
        idx = Person+str(Spec)
#         print(idx)
        try:
            df_concurs_agree[idx] = df_concurs_agree[idx][~df_concurs_agree[idx]["СНИЛС"].isin(to_budget[idx])]
            df_concurs_agree[idx] = df_concurs_agree[idx].reset_index(drop=True)
        except Exception as e:
            str1 = "we have found an exception"
#             print("пропустили", e)
df_concurs_agree["Артем4"].head(15)

In [14]:
df_concurs_agree_head = {}
for Person in Persons:
    for Spec in Persons[Person]["Спец"]:
        idx = Person+str(Spec)
#         print(idx)
        df_concurs_agree_head[idx] = df_concurs_agree[idx][
            df_concurs_agree[idx]["Балл"].astype(int) >= Persons[Person]["Спец"][Spec]["Балл"]
        ]
        df_concurs_agree_head[idx] = df_concurs_agree_head[idx].reset_index(drop=True)
# df_concurs_agree_head["Артем4"]

In [15]:
num_prev = {}
for Person in Persons:
    for Spec in Persons[Person]["Спец"]:
        idx = Person+str(Spec)
#         print(idx)
        m = min(
            df_concurs_agree_head[idx][
                df_concurs_agree_head[idx]["Балл"].astype(int) > Persons[Person]["Спец"][Spec]["Балл"]
            ]["Балл"].astype(int)
        )
        num_prev[idx] = max(df_concurs_agree_head[idx][df_concurs_agree_head[idx]["Балл"].astype(int) == m].index.values)
# num_prev

In [16]:
for Person in Persons:
    for Spec in Persons[Person]["Спец"]:
        idx = Person+str(Spec)
#         print(idx)
        try:
            num = df_concurs_agree[idx][df_concurs_agree[idx]["СНИЛС"] == Persons[Person]["Док"]].index.values[0]
    #         print(num)
            if Spec in [4,5,6]: # просто чтобы поменьше выводилось, все равно бюджет уже проехали
                print(
                    Person + ", " + Persons[Person]["Спец"][Spec]["ОКСО"] + ": \n-\033[1m\033[34m"
                    + str(num + 1)
                    + "\033[0m из "
                    + str(Persons[Person]["Спец"][Spec]["Места"])
                    + " \n-балл предыдущего " + str(df_concurs_agree[idx].iloc[num_prev[idx]]["Балл"])
                    + " \n-всего согласий "
                    + str(len(df_concurs_agree[idx]))
                    + " \n-проходной балл " + str(mark_pass[idx])
                    + " \n (\033[1m\033[34mнаше Согласие тут\033[0m)\n"
                )
        except:
            str1 = "we have found an exception"
            if Spec in [4,5,6]: # просто чтобы поменьше выводилось, все равно бюджет уже проехали
                print(
                    Person + ", " + Persons[Person]["Спец"][Spec]["ОКСО"] + ": \n-\033[1m\033[34m"
                    + str(num_prev[idx] + 2)
                    + "\033[0m из "
                    + str(Persons[Person]["Спец"][Spec]["Места"]) 
                    + " \n-балл предыдущего " + str(df_concurs_agree[idx].iloc[num_prev[idx]]["Балл"])
                    + " \n-всего согласий "
                    + str(len(df_concurs_agree[idx]))
                    + " \n-проходной балл " + str(mark_pass[idx])
                    + " \n (у нас отсутствует Согласие)\n"
                )

Артем, 09.03.01 (платка): 
-15 из 184 
-балл предыдущего 251 
-всего согласий 96 
-проходной балл 171 
 (наше Согласие тут)

Артем, 09.03.03 (платка): 
-9 из 85 
-балл предыдущего 251 
-всего согласий 44 
-проходной балл 191 
 (у нас отсутствует Согласие)

Артем, 10.05.01 (платка): 
-7 из 39 
-балл предыдущего 231 
-всего согласий 19 
-проходной балл 187 
 (у нас отсутствует Согласие)

